## Pandas处理重复值和异常值

In [1]:
import numpy as np
import pandas as pd

### 1、删除重复行

In [2]:
def make_df(indexs, columns):    
    data = [[str(j)+str(i) for j in columns] for i in indexs]
    df = pd.DataFrame(data=data, index=indexs, columns=columns)
    return df

- 使用duplicated()函数检测重复的行
    - 返回元素为布尔类型的Series对象
    - 每个元素对应一行，如果该行不是第一次出现，则元素为True

In [3]:
df = make_df([1, 2, 3, 4], list('ABCD'))
df

,A,B,C,D
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [4]:
# 让第一行和第二行重复
df.loc[1] = df.loc[2]
df

,A,B,C,D
1,A2,B2,C2,D2
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [5]:
# 判断是否和前面的行重复了
df.duplicated()

df.duplicated(keep='first')  # 保留第一行
df.duplicated(keep='last')  # 保留最后一行
df.duplicated(keep=False)  # 标记所有重复行，不保留任何一行


1     True
2     True
3    False
4    False
dtype: bool

In [6]:
df.loc[1, 'D'] = 'DDD'
df

,A,B,C,D
1,A2,B2,C2,DDD
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [7]:
# subset: 子集
df.duplicated(subset=['A','B','C'])

1    False
2     True
3    False
4    False
dtype: bool

- 使用drop_duplicates()函数删除重复的行

In [8]:
df.drop_duplicates()
df.drop_duplicates(subset=['A', 'B', 'C'])
df.drop_duplicates(subset=['A', 'B', 'C'], keep='last')

,A,B,C,D
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


### 2. 映射

映射的含义：创建一个映射关系列表，把values元素和一个特定的标签或者字符串绑定。

包含三种操作：

- replace()函数：替换元素
- map()函数：新建一列, 最重要
- rename()函数：替换索引

#### 1) replace()函数：替换元素

使用replace()函数，对values进行替换操作

In [9]:
index = ['张三', '张三丰', '李白', '杜甫']
columns = ['Python', 'Java', 'H5', "UI"]
data = np.random.randint(0, 100, size=(4, 4))

df = pd.DataFrame(data=data, index=index, columns=columns)
df

,Python,Java,H5,UI
张三,18,99,59,47
张三丰,1,3,78,69
李白,64,47,41,33
杜甫,89,45,41,94


In [10]:
# 替换元素
df.replace({5: 50, 1: 100})

,Python,Java,H5,UI
张三,18,99,59,47
张三丰,100,3,78,69
李白,64,47,41,33
杜甫,89,45,41,94


#### 2) map()函数：适合处理某一单独的列

map()函数中可以使用lambda函数

In [11]:
df2 = df.copy()
df2

,Python,Java,H5,UI
张三,18,99,59,47
张三丰,1,3,78,69
李白,64,47,41,33
杜甫,89,45,41,94


In [12]:
# map一般用在Series数据结构，不能用于DataFrame
# df2['Python'].map({16: 160, 12: 120, 17: 170, 60:600})

# 将Python的每个人的成绩乘以10
df2['Python'].map(lambda x : x * 10)

# 新增一列
df2['Pandas'] = df2['Python'].map(lambda x : x * 10)
df2

,Python,Java,H5,UI,Pandas
张三,18,99,59,47,180
张三丰,1,3,78,69,10
李白,64,47,41,33,640
杜甫,89,45,41,94,890


In [13]:
# 新增一列：判断Java的成绩是否及格
df2['Java是否及格'] = df2['Java'].map(lambda n: '及格'  if n>=60 else '不及格')
df2

,Python,Java,H5,UI,Pandas,Java是否及格
张三,18,99,59,47,180,及格
张三丰,1,3,78,69,10,不及格
李白,64,47,41,33,640,不及格
杜甫,89,45,41,94,890,不及格


In [14]:
# 使用普通函数
# 新增一列： 判断UI成绩
#  <60  不及格
#  60<=n <80 及格
#  >=80 优秀

def fn(n):
    if n < 60:
        return '不及格'
    elif n < 80:
        return '及格'
    return '优秀'

df2['UI等级'] = df2['UI'].map(fn)
df2

,Python,Java,H5,UI,Pandas,Java是否及格,UI等级
张三,18,99,59,47,180,及格,不及格
张三丰,1,3,78,69,10,不及格,及格
李白,64,47,41,33,640,不及格,不及格
杜甫,89,45,41,94,890,不及格,优秀


#### 3) rename()函数：替换索引

In [15]:
df3 = df.copy()
df3

,Python,Java,H5,UI
张三,18,99,59,47
张三丰,1,3,78,69
李白,64,47,41,33
杜甫,89,45,41,94


In [16]:
df3.rename({'张三': 'Mr Zhang'})  # 默认修改行索引名
df3.rename({'Python': '派森'}, axis=1)  # 修改列索引名

df3.rename(index={'张三': 'Mr Zhang'}) # 修改行索引名
df3.rename(columns={'Python': '派森'}) # 修改列索引名

,派森,Java,H5,UI
张三,18,99,59,47
张三丰,1,3,78,69
李白,64,47,41,33
杜甫,89,45,41,94


In [17]:
# 重置索引
df3.reset_index()

,index,Python,Java,H5,UI
0,张三,18,99,59,47
1,张三丰,1,3,78,69
2,李白,64,47,41,33
3,杜甫,89,45,41,94


In [18]:
# 设置行索引
df3.set_index(keys=['H5'])

,Python,Java,UI
H5,,,
59,18,99,47
78,1,3,69
41,64,47,33
41,89,45,94


#### 4) apply()函数：既支持 Series，也支持 DataFrame

In [19]:
df = pd.DataFrame(data=np.random.randint(0, 10, size=(5, 3)),
                              index=list('ABCDE'),
                              columns=['Python', 'NumPy', 'Pandas']
                 )
df

,Python,NumPy,Pandas
A,5,4,9
B,2,7,8
C,6,5,3
D,0,3,0
E,7,2,3


In [20]:
# 用于Series, 其中x表式的Series中元素
df['Python'].apply(lambda x:  True if x>5 else False)

A    False
B    False
C     True
D    False
E     True
Name: Python, dtype: bool

In [21]:
# 用于DataFrame, 其中x是DataFrame中某列或某行的Series数据
df.apply(lambda x : x.mean(), axis=0)  # 求每一列数据的平均值
df.apply(lambda x : x.mean(), axis=1)  # 求每一行数据的平均值

A    6.000000
B    5.666667
C    4.666667
D    1.000000
E    4.000000
dtype: float64

In [22]:
# 自定义方法
def fn2(x):
    return  (np.round(x.mean(), 1), x.count())  # 平均值，计数

df.apply(fn2, axis=1)

A    (6.0, 3)
B    (5.7, 3)
C    (4.7, 3)
D    (1.0, 3)
E    (4.0, 3)
dtype: object

In [23]:
# applymap: DataFrame专有的方法，其中的x是每个元素
df.applymap(lambda x : x + 100) 

,Python,NumPy,Pandas
A,105,104,109
B,102,107,108
C,106,105,103
D,100,103,100
E,107,102,103


#### 5) transform()函数

In [24]:
df = pd.DataFrame(data=np.random.randint(0, 10, size=(5, 3)),
                              index=list('ABCDE'),
                              columns=['Python', 'NumPy', 'Pandas']
                 )
df

,Python,NumPy,Pandas
A,8,0,2
B,9,2,7
C,3,7,9
D,4,6,2
E,0,8,7


In [25]:
# Series中使用transform
# 可以执行多项计算
df['Python'].transform([np.sqrt, np.exp])

# DataFrame中使用transform
def convert(x):
    if x.mean() > 5:
        return x * 10
    return x * (-10)
    
df.transform(convert)  # 处理每一列
df.transform(convert, axis=1)  # 处理每一行

,Python,NumPy,Pandas
A,-80,0,-20
B,90,20,70
C,30,70,90
D,-40,-60,-20
E,0,-80,-70


### 3. 异常值检测和过滤

- describe(): 查看每一列的描述性统计量

In [26]:
df

,Python,NumPy,Pandas
A,8,0,2
B,9,2,7
C,3,7,9
D,4,6,2
E,0,8,7


In [27]:
df.describe()
df.describe([0.01, 0.3, 0.4, 0.9, 0.99])
df.describe([0.01, 0.3, 0.4, 0.9, 0.99]).T

,count,mean,std,min,1%,30%,40%,50%,90%,99%,max
Python,5.0,4.8,3.701351,0.0,0.12,3.2,3.6,4.0,8.6,8.96,9.0
NumPy,5.0,4.6,3.435113,0.0,0.08,2.8,4.4,6.0,7.6,7.96,8.0
Pandas,5.0,5.4,3.209361,2.0,2.00,3.0,5.0,7.0,8.2,8.92,9.0


- df.std() : 可以求得DataFrame对象每一列的标准差

In [28]:
df.std()

Python    3.701351
NumPy     3.435113
Pandas    3.209361
dtype: float64

- df.drop(): 删除特定索引 

In [29]:
df2 = df.copy()
df2

,Python,NumPy,Pandas
A,8,0,2
B,9,2,7
C,3,7,9
D,4,6,2
E,0,8,7


In [30]:
df2.drop('A')  # 默认删除行
df2.drop('Python', axis=1)  # 删除列

df2.drop(index='A')  # 删除行
df2.drop(columns='Python')  # 删除列

# 删除多列或多行
df2.drop(columns=['NumPy', 'Python'])
df2.drop(index=['A', 'B'], inplace=True)
df2

,Python,NumPy,Pandas
C,3,7,9
D,4,6,2
E,0,8,7


- unique() : 唯一,去重

In [31]:
df

,Python,NumPy,Pandas
A,8,0,2
B,9,2,7
C,3,7,9
D,4,6,2
E,0,8,7


In [32]:
# DataFrame没有unique，Series调用unique
df['Python'].unique()

array([8, 9, 3, 4, 0])

- df.query : 按条件查询

In [33]:
df

,Python,NumPy,Pandas
A,8,0,2
B,9,2,7
C,3,7,9
D,4,6,2
E,0,8,7


In [34]:
# ==， >, <
# and, &
#  or ， | 
# in 
df.query('Python == 9')  # 找到Python列中等于9的所有行
df.query('Python < 8') 

df.query('Python>6 and NumPy==2')
df.query('Python>6 & NumPy==2')

df.query('Python==3 or NumPy==2')
df.query('Python==3 | NumPy==2')

df.query('Python in [3, 4, 5, 6]')  # 成员运算符

# 使用变量
n = 7
df.query('Python == @n')  # @n 表式使用变量n的值
m =  [3, 4, 5, 6]
df.query('Python in @m')  # 成员运算符

,Python,NumPy,Pandas
C,3,7,9
D,4,6,2



- df.sort_values(): 根据值排序
- df.sort_index(): 根据索引排序

In [35]:
df

,Python,NumPy,Pandas
A,8,0,2
B,9,2,7
C,3,7,9
D,4,6,2
E,0,8,7


In [36]:
#  sort_values : 默认按照列名排序，默认升序  （常用）
df.sort_values('Python')  
# ascending: 是否升序，默认是True
df.sort_values('Python', ascending=False)  # 降序

# 根据行索引名排序，会把列进行排序（不常用）
df.sort_values('B', axis=1)

,NumPy,Pandas,Python
A,0,2,8
B,2,7,9
C,7,9,3
D,6,2,4
E,8,7,0


In [37]:
df

,Python,NumPy,Pandas
A,8,0,2
B,9,2,7
C,3,7,9
D,4,6,2
E,0,8,7


In [38]:
# 按照索引名排序 （不常用）
# 默认是对行索引进行排序，默认是升序
df.sort_index(ascending=False)
# 按照列索引排序
df.sort_index(ascending=False, axis=1)

,Python,Pandas,NumPy
A,8,2,0
B,9,7,2
C,3,9,7
D,4,2,6
E,0,7,8


- df.info(): 查看数据信息

In [39]:
# info: 常用
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, A to E
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Python  5 non-null      int32
 1   NumPy   5 non-null      int32
 2   Pandas  5 non-null      int32
dtypes: int32(3)
memory usage: 260.0+ bytes


============================================

练习：

    新建一个形状为10000*3的标准正态分布的DataFrame(np.random.randn)，去除掉所有满足以下情况的行：其中任一元素绝对值大于3倍标准差

============================================

In [40]:
df = pd.DataFrame(np.random.randn(10000, 3))
display(df.head())

# 过滤掉 大于3倍标准差的行
# 标准差 df.std()
# 绝对值 df.abs()

# cond：找到每一个元素是否大于3倍标准差
cond = df.abs() > df.std() *3
cond

# 找到存在大于3倍标准差的行
cond2 = cond.any(axis=1)
cond2
# cond2.sum()

# bool值索引，过滤异常值(大于3倍标准差)
df.loc[~cond2]

,0,1,2
0,-0.370802,-1.115758,-0.154760
1,-1.117441,0.771005,0.582512
2,0.395195,-0.058415,-1.107285
3,1.404234,1.829916,0.457755
4,-0.509765,-0.989182,1.520186


,0,1,2
0,-0.370802,-1.115758,-0.154760
1,-1.117441,0.771005,0.582512
2,0.395195,-0.058415,-1.107285
3,1.404234,1.829916,0.457755
4,-0.509765,-0.989182,1.520186
...,...,...,...
9995,0.988193,-0.759094,0.187001
9996,-1.165700,0.902860,0.864895
9997,1.144086,0.083804,0.375425
9998,0.430814,-0.689308,-1.578386


### 4. 抽样

- 使用.take()函数排序
- 可以借助np.random.permutation()函数随机排序

无放回抽样

In [41]:
df2

,Python,NumPy,Pandas
C,3,7,9
D,4,6,2
E,0,8,7


In [42]:
df2.take([1, 0, 2])  # 行排列
df2.take([1, 0, 2], axis=1)  # 列排列

# 随机排列
np.random.permutation([0, 1, 2])

# 无放回抽样: 依次随机取出，没有重复值
df2.take(np.random.permutation([0, 1, 2]))

,Python,NumPy,Pandas
D,4,6,2
E,0,8,7
C,3,7,9


有放回抽样

In [43]:
# 有放回抽样： 可能会出现重复值

np.random.randint(0, 3, size=5)

df2.take(np.random.randint(0, 3, size=5))

,Python,NumPy,Pandas
E,0,8,7
E,0,8,7
E,0,8,7
D,4,6,2
D,4,6,2
